# Generative Grammar

The purpose of this notebook is to explain the use of prograbilistic contex free grammars to increase the size and diversity of our training dataset for the supervised classification task. Specifically, we will expand on:

1. The purpose of using a prograbilistic contex free grammar as a data augmentation tool
2. How we encode student programs in the grammar, specifically the main function structure, simple and complex syntax modifications, as well as rare programs.
3. The assumptions that this tool holds.

The complete code for the generative grammar is in https://github.com/willcrichton/autoplan/blob/master/grammars/rainfall/ocaml.py.

## Probabilistic Context Free Grammars as a Data Augmentation Tool

We discussed in the "Supervised Simple Classifier Ablation and Parameter Sweep" notebook that the classification task can reach up to 85% accuracy when given ideal hyperparameters. However, the supervised classification task performed on that notebook was heavily dependent on labeled student data, which is limited and has a high human cost. In attempting to address this issue, we study the use of probabilistic context free grammars to increase the size and diversity of our training dataset. Given a small set of human-labeled student data, we encode semantically equivalent syntactic variations of code, maintaining the overall structure. 

For example, see below the first non-terminal for the OCaml grammar that separates programs by their plans: 

```ocaml
class Program(Rule):
    def render(self):
        strategy = self.choice('strategy', {
            Labels.SingleLoop: 1, (* The options are given weights. *)
            Labels.CleanFirst: 1,
            Labels.CleanInSC: 1 (* CleanInSC refers to the Clean Multiple plan *)
        })

        self.set_label(int(strategy))

        (* Once a choice is made, the node is parsed accordingly. *)
        if strategy == Labels.SingleLoop:
            return SingleLoop().render()
        if strategy == Labels.CleanFirst:
            return CleanFirst().render()
        elif strategy == Labels.CleanInSC:
            return CleanInSC().render()
```

## Main Function, Sound Variations, and Rare Programs

Following this top-down approach, we encode further program choices starting from the main function. For example, see below the main Single Loop function. Upon analysing student data, we identified that the main variations at this point steemed from:

1. Deciding weather or not to write a recursive function;
2. Deciding weather or not to specify variable types in the function signature;
3. Deciding weather or not to use in-body helper functions. 

Thus, we add choices for `recursion`, `params`, and `helper_in_body` accordingly: 

### Main Function

```ocaml
{%- if not helper_in_body -%}
    {{helper_body}}
{%- endif -%}

let {{recursion}} rainfall {{params}}
    {%- if helper_in_body -%} (* helper body + rainfall body *)
    {{helper_body}} in 
    {{rainfall_body}}
    {% else -%} (* rainfall body for out of body helpers *)
    {{rainfall_body}}
    {% endif -%}
```

Note that the main function for Clean First and Clean Multiple follow a similar structure. 

### Sound Variations

#### Simple Variations

To understand the encoding syntactic variations of the function signature, see below the `params` node. It relies on a boolean variable `uses_annotation` that we encoded. Note that both options are semantically equivalent. 

```ocaml
{%- set params -%}
    {%- if uses_annotation -%} (* Add types *)
        (list_name : {{_type}} list) : {{_type}} = 
    {%- else -%} (* Exclude types *) 
        list_name = 
    {%- endif -%}
{%- endset -%}
```

#### Complex Variations

Now, take a closer look a the `rainfall_body` node in the Single Loop structure. Once again, it encodes all syntactically equivalent options students may choose when writing this part of their solution. 

```ocaml
{%- set rainfall_body -%}
    (* Students may choose to directly call their helper function, 
       without checking for division by zero first. 
       The helper function is responsible for outputting the average. *)
    {%- if rainfall_body_specs == 'direct_pass' -%} 
        helper_name list_name 0{{dot}} 0{{dot}}
    
    (* Students may choose to call their helper function using
       `match` or `let`, and having checked for division by zero.
       The helper function is responsible for outputting the average. *)
    {%- elif recursion_strategy == 'match' -%} {# recurse #}
        match list_name with
        | {{check_empty_list}} -> {{failure}}
        | _ -> helper_name list_name 0{{dot}} 0{{dot}}
    
    {%- elif recursion_strategy == 'let' -%}
        let (addition_var, counter_var) = helper_name list_name 0{{dot}} 0{{dot}} in 
        if counter_var = 0{{dot}} then {{failure}}
        else (addition_var /{{dot}} counter_var)
    
    (* Finally, students can choose to output the average in the main function. *)
    {%- else -%}
        match list_name with
        | {{check_empty_list}} -> {{failure}}
        | (addition_var, counter_var) -> addition_var /{{dot}} counter_var
    {%- endif -%}
{%- endset -%}
```

### Rare Programs

Finally, in order to encode rare structures we can add nodes that are activated only under specific conditions. For example, in the node below, the code is only parsed if the variable `anonymous_helpers` is `True`. Only one student wrote this structure as part of their Clean Multiple solution, but the grammar can now both parse its structure in multiple programs as well as encode further variations, such as the different ways to output `failure` to compute the average.  

```ocaml
(...)

{% elif anonymous_helpers -%} 
(try ((List.fold_right 
(fun var var -> (if (var = (-999)) then {{failure}} else if (var < 0) then var 
 else (var + var))) list_name 0) /{{dot}} (List.fold_right 
    (fun var var -> (if (var = (-999)) then {{failure}} else if (var < 0) then var 
     else (1 + var))) list_name 0)) with division_by_zero_helper_name -> {{failure}})
{% endif -%}

(...)
```

## Discussion

This generative tool holds two main assumptions: 

1. A human expert can correctly classify strategies of a small set (~30 samples) of student solutions. 
2. A human expert can correctly encode sound variarions (semantically equivalent) of lines of code for a given task, 

where a human expert can be a faculty member or a teaching assistant. 

With that, we were able to generate 10k+ unique solutions, far more than the 44 programs we analyzed, increasing the size and diversity of our training dataset by 227 fold. 